In [ ]:
# useful Notebooks


In [ ]:
 ''' From Statistical Testing Notebook, we need this for mean-difference tests
     (Fri -> Mon, ToM vs non-ToM, January vs Rest, H1 vs H2, Weekday pairs, etc.). '''
import numpy as np
from scipy import stats

def manual_t_from_summary(mean_a, mean_b, var_a, var_b, n_a, n_b, two_sided=True):
    """
    Two-sample t-test from SUMMARY STATS (equal-variance classical form).
    Use when you already have group means/variances/N (e.g., pre-aggregated by weekday).
    Parameters are sample variances (ddof=1).
    """
    # pooled standard error under equal-variance assumption
    se = np.sqrt(var_a / n_a + var_b / n_b)
    t_stat = (mean_a - mean_b) / se
    df = n_a + n_b - 2
    # two-sided p-value by default
    if two_sided:
        p = 2 * (1 - stats.t.cdf(np.abs(t_stat), df=df))
    else:
        p = 1 - stats.t.cdf(t_stat, df=df)
    return t_stat, df, p

def manual_t_from_samples(a, b, equal_var=True, two_sided=True):
    """
    Two-sample t-test from RAW SAMPLES (arrays of returns).
    Matches the structure of the original cell but operates on your empirical groups.
    """
    n_a, n_b = len(a), len(b)
    mean_a, mean_b = np.mean(a), np.mean(b)
    var_a = np.var(a, ddof=1)
    var_b = np.var(b, ddof=1)

    if equal_var:
        # classical pooled-variance t
        se = np.sqrt(var_a / n_a + var_b / n_b)
        df = n_a + n_b - 2
    else:
        # Welch's t (safer when variances differ)
        se = np.sqrt(var_a / n_a + var_b / n_b)
        df = (var_a / n_a + var_b / n_b) ** 2 / (
            (var_a**2) / (n_a**2 * (n_a - 1)) + (var_b**2) / (n_b**2 * (n_b - 1))
        )

    t_stat = (mean_a - mean_b) / se
    if two_sided:
        p = 2 * (1 - stats.t.cdf(np.abs(t_stat), df=df))
    else:
        p = 1 - stats.t.cdf(t_stat, df=df)
    return t_stat, df, p

def scipy_t_from_samples(a, b, equal_var=True):
    """
    SciPy wrapper equivalent of the above (useful for quick checks).
    """
    t_stat, p = stats.ttest_ind(a, b, equal_var=equal_var)
    # SciPy returns two-sided p; df is not returned, so we compute it if equal_var, else Welch-approx.
    n_a, n_b = len(a), len(b)
    if equal_var:
        df = n_a + n_b - 2
    else:
        var_a = np.var(a, ddof=1); var_b = np.var(b, ddof=1)
        df = (var_a / n_a + var_b / n_b) ** 2 / (
            (var_a**2) / (n_a**2 * (n_a - 1)) + (var_b**2) / (n_b**2 * (n_b - 1))
        )
    return t_stat, df, p

# ---- Example usage (REPLACE with your actual arrays) ----
# fri_returns = ...   # numpy array
# mon_returns = ...   # numpy array
# t, df, p = manual_t_from_samples(fri_returns, mon_returns, equal_var=False)
# print(f"Fri−Mon: t={t:.3f}, df={df:.1f}, p={p:.3g}")


# ''' From Statistical Testing Notebook, we need this for global multi-group tests
#     (Month-of-Year bars Jan…Dec → overall ANOVA; optionally Weekday 5-group test). '''
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

def anova_from_groups(group_dict, label_col="group", value_col="value"):
    """
    One-way ANOVA using statsmodels, mirroring your original notebook pattern
    but without loading any external demo file.
    - group_dict: {'Jan': array_like, 'Feb': array_like, ..., 'Dec': array_like}
                  (or {'Mon':..., 'Tue':..., ...} for weekday)
    Returns (f_value, p_value, anova_table)
    """
    # Build long DataFrame like your melt step
    rows = []
    for g, arr in group_dict.items():
        arr = np.asarray(arr)
        rows.append(pd.DataFrame({label_col: g, value_col: arr}))
    df_long = pd.concat(rows, ignore_index=True)

    # OLS with categorical factor
    model = ols(f"{value_col} ~ C({label_col})", data=df_long).fit()
    anova_tbl = sm.stats.anova_lm(model, typ=2)
    f_value = float(anova_tbl.loc[f"C({label_col})", "F"])
    p_value = float(anova_tbl.loc[f"C({label_col})", "PR(>F)"])
    return f_value, p_value, anova_tbl

# ---- Example usage (REPLACE with your month buckets) ----
# month_groups = {
#     'Jan': returns_jan, 'Feb': returns_feb, 'Mar': returns_mar, 'Apr': returns_apr,
#     'May': returns_may, 'Jun': returns_jun, 'Jul': returns_jul, 'Aug': returns_aug,
#     'Sep': returns_sep, 'Oct': returns_oct, 'Nov': returns_nov, 'Dec': returns_dec
# }
# F, p, table = anova_from_groups(month_groups, label_col="month", value_col="r")
# print(f"MoY ANOVA: F={F:.3f}, p={p:.3g}")
# display(table)


In [ ]:
# ''' From Linear Regression Lab, we need this for:
#      (1) running OLS with a formula on our returns using calendar dummies/controls, and
#      (2) basic diagnostics + a tidy coefficient table with 95% CI for plotting betas. '''
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

def fit_ols(formula: str, data: pd.DataFrame):
    """
    Fit OLS via statsmodels using a Patsy formula (e.g.,
    'r ~ C(weekday) + ToM + C(month) + log_size + C(sector)').
    Returns the fitted model.
    """
    model = smf.ols(formula=formula, data=data).fit()
    return model

def coef_table_with_ci(model, alpha: float = 0.05) -> pd.DataFrame:
    """
    Return a tidy coefficient table: term, estimate, std err, t, p, [CI_low, CI_high].
    Useful for 'calendar-dummy regression betas ± CI' plots.
    """
    params = model.params.rename("estimate")
    bse = model.bse.rename("std_err")
    tvals = model.tvalues.rename("t")
    pvals = model.pvalues.rename("p")
    ci = model.conf_int(alpha=alpha)
    ci.columns = ["ci_low", "ci_high"]
    out = pd.concat([params, bse, tvals, pvals, ci], axis=1).reset_index()
    out = out.rename(columns={"index": "term"})
    return out

def plot_residuals_vs_fitted(model, title: str = None):
    """
    Basic diagnostic: residuals vs fitted. Helps check mean-variance patterns
    and outliers after calendar-dummy regressions.
    """
    resid = model.resid
    fitted = model.fittedvalues
    plt.figure()
    plt.scatter(fitted, resid, alpha=0.7)
    plt.axhline(0, linestyle="--")
    plt.xlabel("Fitted values")
    plt.ylabel("Residuals")
    plt.title(title or "Residuals vs Fitted")
    plt.show()

# ---- Example usage (REPLACE with your real DataFrame/columns) ----
# df = ...  # your returns with engineered features (weekday, ToM, month, etc.)
# model = fit_ols("r ~ C(weekday) + ToM + C(month) + C(quarter_end) + log_size + C(sector)", data=df)
# betas = coef_table_with_ci(model, alpha=0.05)
# display(betas)
# plot_residuals_vs_fitted(model, title="Residuals vs Fitted (calendar-dummy OLS)")


In [ ]:
# ''' From MLR-Practice Notebook, we need this for sector-controlled calendar regressions:
#     - OLS with Patsy formula: r ~ calendar dummies + controls + C(sector) (fixed effects)
#     - Variance Inflation Factor (VIF) to check multicollinearity among numeric controls
#     - Durbin–Watson for autocorrelation diagnostic
#     - Residuals vs Fitted plot (basic heteroskedasticity/outlier check)
#     This supports: "Size-/sector-controlled coefficient plot (calendar-dummy regression betas ± CI)". '''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
from patsy import dmatrices, dmatrix

# -------- OLS wrapper (sector FE via C(sector)) --------
def fit_ols(formula: str, data: pd.DataFrame):
    """
    Fit OLS with a Patsy formula on your returns DataFrame.
    Example formula:
      r ~ C(weekday) + ToM + C(month) + log_size + C(sector)
    """
    return smf.ols(formula=formula, data=data).fit()

# -------- Tidy coefficient table (for betas ± CI plots) --------
def coef_table_with_ci(model, alpha: float = 0.05) -> pd.DataFrame:
    params = model.params.rename("estimate")
    bse = model.bse.rename("std_err")
    tvals = model.tvalues.rename("t")
    pvals = model.pvalues.rename("p")
    ci = model.conf_int(alpha=alpha)
    ci.columns = ["ci_low", "ci_high"]
    out = pd.concat([params, bse, tvals, pvals, ci], axis=1).reset_index()
    return out.rename(columns={"index": "term"})

# -------- Residuals vs Fitted diagnostic --------
def plot_residuals_vs_fitted(model, title: str = "Residuals vs Fitted"):
    resid = model.resid
    fitted = model.fittedvalues
    plt.figure()
    plt.scatter(fitted, resid, alpha=0.7)
    plt.axhline(0, linestyle="--")
    plt.xlabel("Fitted values")
    plt.ylabel("Residuals")
    plt.title(title)
    plt.show()

# -------- VIF for numeric columns in a formula context --------
def compute_vif_from_formula(formula: str, data: pd.DataFrame) -> pd.DataFrame:
    """
    Computes VIF on the numeric design matrix implied by `formula`.
    Categorical terms (e.g., C(sector), C(weekday)) expand to dummies; VIF is computed
    on the full exogenous matrix excluding the intercept.
    """
    # Build design matrices (y unused here, but y/X ensures same design handling as the model)
    y, X = dmatrices(formula, data=data, return_type="dataframe")
    # Drop intercept if present
    if "Intercept" in X.columns:
        X_no_intercept = X.drop(columns=["Intercept"])
    else:
        X_no_intercept = X.copy()
    vif_rows = []
    X_vals = X_no_intercept.values
    for i, col in enumerate(X_no_intercept.columns):
        vif_val = variance_inflation_factor(X_vals, i)
        vif_rows.append({"term": col, "VIF": float(vif_val)})
    return pd.DataFrame(vif_rows).sort_values("VIF", ascending=False).reset_index(drop=True)

# -------- Durbin–Watson convenience wrapper --------
def durbin_watson_stat(model) -> float:
    """DW statistic ~2 means low first-order autocorrelation."""
    return float(durbin_watson(model.resid))

# ---------------- Example usage (REPLACE with your real DataFrame/columns) ----------------
# df = ...  # Your panel of returns with columns: r, weekday/month/ToM flags, sector, optional log_size, etc.

# Example sector-controlled calendar regression:
# formula = "r ~ C(weekday) + ToM + C(month) + log_size + C(sector)"
# mdl = fit_ols(formula, df)
# betas = coef_table_with_ci(mdl)
# print(mdl.summary().tables[0])        # brief header
# display(betas)                        # tidy table for plotting betas ± CI
# print('Durbin–Watson:', durbin_watson_stat(mdl))
# vif_tbl = compute_vif_from_formula(formula, df)
# display(vif_tbl)
# plot_residuals_vs_fitted(mdl, title="Residuals vs Fitted (calendar + sector FE)")


In [ ]:
''' From Logistic-EDA ideas (repurposed for our calendar-effects QC):
    - Group-size bar charts for calendar buckets (e.g., Weekday, ToM vs non-ToM, H1 vs H2, Quarter-end vs other)
    - Correlation heatmap for numeric controls (pairs naturally with our VIF table)
  Why: show sample sizes (assumptions/context for t/ANOVA) and inspect collinearity among controls before OLS.
  Uses only numpy/pandas/matplotlib (no seaborn). Ready to drop into our project notebook. '''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- Group-size tables & bars ----------

def group_size_table(df: pd.DataFrame, group_col: str) -> pd.DataFrame:
    """
    Returns a small table with counts and percentages by group.
    Useful to place next to mean-difference bars so readers see n per bucket.
    """
    vc = df[group_col].value_counts(dropna=False)
    tbl = pd.DataFrame({
        group_col: vc.index.astype(object),
        "n": vc.values
    })
    tbl["pct"] = (tbl["n"] / tbl["n"].sum()) * 100.0
    return tbl.sort_values(group_col).reset_index(drop=True)

def plot_group_size_bar(
    df: pd.DataFrame,
    group_col: str,
    title: str = None,
    order: list | None = None,
    annotate: bool = True
):
    """
    Bar chart of sample counts per group_col.
    - order: optional list specifying the display order (e.g., ["Mon","Tue","Wed","Thu","Fri"])
    - annotate: add 'n (pct%)' labels above bars
    """
    tbl = group_size_table(df, group_col)
    if order is not None:
        # Ensure all categories present; if not, we still show with zero counts
        all_vals = pd.Index(order, name=group_col)
        tbl = tbl.set_index(group_col).reindex(all_vals, fill_value=0).reset_index()
    x = tbl[group_col].astype(str).tolist()
    y = tbl["n"].to_numpy()
    pct = tbl["pct"].to_numpy()

    fig, ax = plt.subplots()
    bars = ax.bar(x, y)
    ax.set_xlabel(group_col)
    ax.set_ylabel("Sample count (n)")
    ax.set_title(title or f"Group sizes: {group_col}")
    ax.set_ylim(0, max(y) * 1.15 if len(y) else 1)

    if annotate:
        for rect, n_i, p_i in zip(bars, y, pct):
            ax.text(
                rect.get_x() + rect.get_width() / 2.0,
                rect.get_height(),
                f"{int(n_i)} ({p_i:.1f}%)",
                ha="center", va="bottom", rotation=0
            )
    plt.show()
    return tbl  # so you can display the table beside the plot

# ---------- Correlation heatmap for numeric controls ----------

def correlation_df(
    df: pd.DataFrame,
    numeric_cols: list[str] | None = None,
    method: str = "pearson"
) -> pd.DataFrame:
    """
    Compute correlation matrix among numeric controls.
    - numeric_cols: explicit list; if None, auto-detect numeric columns.
    - method: 'pearson' (default), 'spearman', or 'kendall'
    """
    if numeric_cols is None:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    corr = df[numeric_cols].corr(method=method)
    return corr

def plot_correlation_heatmap(
    corr: pd.DataFrame,
    title: str = "Correlation heatmap (controls)",
    annotate: bool = True
):
    """
    Plot a heatmap (matplotlib only) from a correlation DataFrame.
    - annotate: write correlation values in each cell
    """
    labels = corr.columns.tolist()
    fig, ax = plt.subplots()
    im = ax.imshow(corr.to_numpy(), aspect="auto", origin="upper")
    ax.set_xticks(range(len(labels)))
    ax.set_yticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha="right")
    ax.set_yticklabels(labels)
    ax.set_title(title)
    cbar = fig.colorbar(im, ax=ax)
    cbar.ax.set_ylabel("corr", rotation=90)

    if annotate:
        mat = corr.to_numpy()
        nrows, ncols = mat.shape
        for i in range(nrows):
            for j in range(ncols):
                ax.text(j, i, f"{mat[i, j]:.2f}", ha="center", va="center")

    plt.tight_layout()
    plt.show()

# ---------------- Example usage (REPLACE with your real DataFrame/columns) ----------------
# df = ...  # your cleaned returns DataFrame with calendar flags/controls
#
# # 1) Group-size bars (QC next to mean-difference charts):
# plot_group_size_bar(df, "weekday", order=["Mon","Tue","Wed","Thu","Fri"])
# plot_group_size_bar(df, "ToM", title="Group sizes: Turn-of-the-Month (ToM)")
# plot_group_size_bar(df, "half", order=["H1","H2"], title="Group sizes: H1 vs H2")
# plot_group_size_bar(df, "quarter_end", title="Group sizes: Quarter-end vs other")
#
# # 2) Correlation heatmap for numeric controls (pair with our VIF table):
# num_cols = ["log_size", "volatility_20d", "liquidity", "beta"]  # example
# corr = correlation_df(df, numeric_cols=num_cols, method="pearson")
# plot_correlation_heatmap(corr, title="Controls correlation (Pearson)")
#
# # VIF remains in our existing helper:
# # from earlier cell:
# # vif_tbl = compute_vif_from_formula("r ~ C(weekday) + ToM + C(month) + log_size + C(sector)", df)
# # display(vif_tbl)


In [ ]:
# ''' From Paired-Tests Notebook, we need this for:
#     - Overnight vs Intraday comparisons on the SAME dates (paired samples)
#     - Optional H1 vs H2 if computed as per-year pairs (e.g., mean(H1_year_i) vs mean(H2_year_i))
#   Outputs t, df, p and 95% CI for the mean difference; also a grouped version (e.g., by weekday).
# '''

import numpy as np
import pandas as pd
from scipy import stats

def paired_t_from_samples(a, b, alpha: float = 0.05, two_sided: bool = True):
    """
    Paired t-test on two same-length arrays a, b (pair matched row-wise).
    Returns dict with: n, mean_a, mean_b, mean_diff, se_diff, t, df, p, ci_low, ci_high.
    Drops pairs with any NaN.
    """
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    mask = np.isfinite(a) & np.isfinite(b)
    a = a[mask]; b = b[mask]
    n = a.size
    if n < 2:
        return {
            "n": n, "mean_a": np.nan, "mean_b": np.nan, "mean_diff": np.nan,
            "se_diff": np.nan, "t": np.nan, "df": np.nan, "p": np.nan,
            "ci_low": np.nan, "ci_high": np.nan
        }
    d = a - b
    mean_a = float(np.mean(a))
    mean_b = float(np.mean(b))
    mean_diff = float(np.mean(d))
    sd_diff = float(np.std(d, ddof=1))
    se_diff = sd_diff / np.sqrt(n)
    df = n - 1
    t_stat = mean_diff / se_diff if se_diff > 0 else np.inf
    if two_sided:
        p = 2 * (1 - stats.t.cdf(np.abs(t_stat), df=df))
        tcrit = stats.t.ppf(1 - alpha/2, df=df)
    else:
        p = 1 - stats.t.cdf(t_stat, df=df)
        tcrit = stats.t.ppf(1 - alpha, df=df)
    half_width = tcrit * se_diff
    ci_low = mean_diff - half_width
    ci_high = mean_diff + half_width
    return {
        "n": int(n),
        "mean_a": mean_a,
        "mean_b": mean_b,
        "mean_diff": mean_diff,
        "se_diff": se_diff,
        "t": float(t_stat),
        "df": float(df),
        "p": float(p),
        "ci_low": float(ci_low),
        "ci_high": float(ci_high),
    }

def paired_tests_by_group(
    df: pd.DataFrame,
    group_col: str,
    a_col: str,
    b_col: str,
    order: list | None = None,
    alpha: float = 0.05,
    two_sided: bool = True
) -> pd.DataFrame:
    """
    Run paired t-tests within each group (e.g., by 'weekday').
    Assumes rows are date-aligned so (a_col, b_col) are valid pairs within each group slice.
    Returns tidy DataFrame: [group, n, mean_a, mean_b, mean_diff, ci_low, ci_high, t, df, p].
    """
    results = []
    # Determine group order
    groups = order if order is not None else (
        df[group_col].dropna().unique().tolist()
    )
    for g in groups:
        sub = df[df[group_col] == g]
        res = paired_t_from_samples(sub[a_col].values, sub[b_col].values, alpha=alpha, two_sided=two_sided)
        row = {"group": g}; row.update(res)
        results.append(row)
    out = pd.DataFrame(results)
    # If some groups in order had no rows, fill with NaNs already produced by the function (n=0)
    return out

# ---------------- Example usage (REPLACE with your real columns) ----------------
# df = ...  # cleaned returns with columns: date, weekday, r_overnight, r_intraday, ...
#
# # Overall paired test (all dates):
# overall = paired_t_from_samples(df["r_overnight"], df["r_intraday"], alpha=0.05, two_sided=True)
# print("Overnight vs Intraday (overall):", overall)
#
# # By-weekday paired tests (to annotate 'Overnight vs Intraday by weekday' bars):
# wd_order = ["Mon","Tue","Wed","Thu","Fri"]
# by_wd = paired_tests_by_group(df, group_col="weekday",
#                               a_col="r_overnight", b_col="r_intraday",
#                               order=wd_order, alpha=0.05, two_sided=True)
# display(by_wd)
#
# # If doing H1 vs H2 paired by year (per-year means), first build per-year pairs
# # year_means = df.groupby(["year","half"], as_index=False)["r"].mean().pivot(index="year", columns="half", values="r")
# # h1 = year_means["H1"].values; h2 = year_means["H2"].values
# # print(paired_t_from_samples(h2, h1, alpha=0.05, two_sided=True))  # (H2 - H1)


In [ ]:
# ''' From Discretization Notes (3-4-5 rule), adapted for calendar-effects cross-sections:
#     - three_four_five_bins(x): bin a numeric control (e.g., log_size, volatility) into natural intervals
#     - bucket_diff_bar(...): per-bucket mean difference (group A vs group B) with 95% CI bars
#   Use cases:
#     * ToM premium by SIZE bucket (bars with CI, t/p optional)
#     * Weekday mean by VOLATILITY bucket (5 bars × 5 buckets) or heatmap
#   Plotting: matplotlib only.
# '''

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

def _rule_edges(n_msd: int, lo: float, hi: float) -> list[float]:
    """Return bin edges per 3-4-5 rule for integer n_msd in {1..10}."""
    width = (hi - lo)
    if n_msd in (3, 6, 9):
        k = 3; gaps = [i*width/k for i in range(k+1)]
        return [lo + g for g in gaps]
    if n_msd == 7:
        # 2-3-2 split
        g = width / 7.0
        return [lo, lo+2*g, lo+5*g, hi]
    if n_msd in (2, 4, 8):
        k = 4; gaps = [i*width/k for i in range(k+1)]
        return [lo + g for g in gaps]
    if n_msd in (1, 5, 10):
        k = 5; gaps = [i*width/k for i in range(k+1)]
        return [lo + g for g in gaps]
    # Fallback: single bin
    return [lo, hi]

def three_four_five_bins(x: pd.Series, trim_pct: float = 0.0, label_prefix: str = "") -> pd.Categorical:
    """
    Discretize numeric Series x using the 3-4-5 rule.
    - trim_pct: e.g., 0.05 to drop bottom/top 5% before setting the outer edges (default 0 for cleaned data)
    - returns a pandas.Categorical with (a, b] style labels and stores edges in .attrs['edges']
    """
    s = pd.to_numeric(x, errors="coerce")
    s_valid = s.dropna()
    if trim_pct > 0:
        lo_q = s_valid.quantile(trim_pct)
        hi_q = s_valid.quantile(1 - trim_pct)
        s_clip = s_valid[(s_valid >= lo_q) & (s_valid <= hi_q)]
    else:
        s_clip = s_valid

    if s_clip.empty:
        return pd.Categorical([np.nan]*len(s), categories=[])

    # Most significant digit scale
    max_abs = float(np.max(np.abs([s_clip.min(), s_clip.max()])))
    j = int(math.floor(math.log10(max_abs))) if max_abs > 0 else 0
    scale = 10.0**j

    lo = math.floor(s_clip.min() / scale) * scale
    hi = math.ceil(s_clip.max() / scale) * scale
    n_msd = int(round((hi - lo) / scale))
    n_msd = max(1, min(10, n_msd))  # clamp

    edges = _rule_edges(n_msd, lo, hi)
    # Ensure strictly increasing edges
    edges = [edges[0]] + [edges[i] if edges[i] > edges[i-1] else edges[i-1] + np.finfo(float).eps
                          for i in range(1, len(edges))]

    cats = pd.cut(s, bins=edges, right=True, include_lowest=False)
    if label_prefix:
        cats = cats.rename_categories(lambda lab: f"{label_prefix} {lab}")
    # keep edges for reuse
    cats = cats.set_attrs({"edges": edges})
    return cats

def _mean_ci(a: np.ndarray, alpha: float = 0.05):
    a = a[np.isfinite(a)]
    n = a.size
    m = float(np.mean(a)) if n else np.nan
    if n <= 1:
        return m, (np.nan, np.nan), n
    s = float(np.std(a, ddof=1))
    tcrit = stats.t.ppf(1 - alpha/2, df=n-1)
    half = tcrit * s / np.sqrt(n)
    return m, (m - half, m + half), n

from scipy import stats

def bucket_diff_bar(
    df: pd.DataFrame,
    bucket_col: str,
    value_col: str,
    group_col: str,
    group_a, group_b,
    alpha: float = 0.05,
    title: str | None = None
):
    """
    For each bucket, compute mean(value | group==A) - mean(value | group==B) with 95% CI (Welch).
    Displays a bar chart with CI whiskers; returns a tidy results DataFrame.
    """
    buckets = df[bucket_col].cat.categories if isinstance(df[bucket_col].dtype, pd.CategoricalDtype) else sorted(df[bucket_col].dropna().unique())
    rows = []
    diffs, ci_l, ci_u, labels = [], [], [], []

    for b in buckets:
        sub = df[df[bucket_col] == b]
        a = sub.loc[sub[group_col] == group_a, value_col].to_numpy(dtype=float)
        bvals = sub.loc[sub[group_col] == group_b, value_col].to_numpy(dtype=float)

        # Welch stats
        na, nb = len(a), len(bvals)
        ma, mb = np.mean(a) if na else np.nan, np.mean(bvals) if nb else np.nan
        va, vb = np.var(a, ddof=1) if na > 1 else np.nan, np.var(bvals, ddof=1) if nb > 1 else np.nan
        se = np.sqrt(va/na + vb/nb) if na and nb and na>1 and nb>1 else np.nan
        t = (ma - mb) / se if se and np.isfinite(se) else np.nan
        df_w = (va/na + vb/nb)**2 / ((va**2)/(na**2*(na-1)) + (vb**2)/(nb**2*(nb-1))) if na>1 and nb>1 else np.nan
        p = 2*(1 - stats.t.cdf(np.abs(t), df=df_w)) if np.isfinite(t) and np.isfinite(df_w) else np.nan

        # CI for difference
        if na>1 and nb>1 and np.isfinite(df_w):
            tcrit = stats.t.ppf(1 - alpha/2, df=df_w)
            half = tcrit * se
            ci = ( (ma-mb) - half, (ma-mb) + half )
        else:
            ci = (np.nan, np.nan)

        rows.append({
            "bucket": str(b),
            "n_A": int(na), "mean_A": float(ma),
            "n_B": int(nb), "mean_B": float(mb),
            "diff_A_minus_B": float(ma - mb),
            "ci_low": float(ci[0]), "ci_high": float(ci[1]),
            "t": float(t) if np.isfinite(t) else np.nan,
            "df": float(df_w) if np.isfinite(df_w) else np.nan,
            "p": float(p) if np.isfinite(p) else np.nan
        })

        labels.append(str(b))
        diffs.append(ma - mb)
        ci_l.append((ma - mb) - ci[0] if np.isfinite(ci[0]) else 0.0)
        ci_u.append(ci[1] - (ma - mb) if np.isfinite(ci[1]) else 0.0)

    res = pd.DataFrame(rows)

    # Plot
    x = np.arange(len(labels))
    fig, ax = plt.subplots()
    bars = ax.bar(x, diffs, yerr=[ci_l, ci_u], capsize=4)
    ax.set_xticks(x); ax.set_xticklabels(labels, rotation=45, ha="right")
    ax.set_ylabel(f"Mean({value_col} | {group_col}={group_a}) - Mean({value_col} | {group_col}={group_b})")
    ax.set_title(title or f"Difference by {bucket_col}")
    plt.tight_layout(); plt.show()
    return res

# ---------------- Example usage (REPLACE with your real DataFrame/columns) ----------------
# df = ...  # cleaned returns with columns like: r, ToM (bool or {1,0}), weekday, log_size, volatility_20d, ...
#
# # A) Create SIZE buckets via 3-4-5 rule (no trimming since data are cleaned):
# df["size_bucket"] = three_four_five_bins(df["log_size"], trim_pct=0.0, label_prefix="Size")
#
# # B) ToM premium by SIZE bucket (A=ToM, B=non-ToM):
# tom_by_size = bucket_diff_bar(df, bucket_col="size_bucket", value_col="r",
#                               group_col="ToM", group_a=1, group_b=0,
#                               title="ToM premium by size bucket (Δ ±95% CI)")
# # C) Weekday mean by VOLATILITY bucket can be built by running bucketed means per weekday, or a heatmap.


In [ ]:
# ''' VIF visual: compact bar chart to accompany compute_vif_from_formula(...)
#     - Input: the DataFrame returned by compute_vif_from_formula(formula, df)
#     - Purpose: quickly spot multicollinearity (e.g., VIF > 5 or > 10) in our calendar-dummy OLS
#     - Plotting: matplotlib only (no seaborn)
# '''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def plot_vif_bar(vif_df: pd.DataFrame, threshold: float = 5.0, top_n: int | None = None, title: str | None = None):
    """
    Plot VIF values as a horizontal bar chart.
    Parameters
    ----------
    vif_df : DataFrame with columns ['term','VIF'] as returned by compute_vif_from_formula(...)
    threshold : float, draw a vertical reference line at this VIF (e.g., 5 or 10)
    top_n : show only the top_n terms by VIF (None = all)
    title : optional title
    """
    df_plot = vif_df.copy()
    if top_n is not None:
        df_plot = df_plot.sort_values("VIF", ascending=False).head(top_n)
    else:
        df_plot = df_plot.sort_values("VIF", ascending=True)

    terms = df_plot["term"].tolist()
    vals = df_plot["VIF"].to_numpy()

    y = np.arange(len(terms))
    fig, ax = plt.subplots()
    ax.barh(y, vals)
    ax.set_yticks(y)
    ax.set_yticklabels(terms)
    ax.set_xlabel("VIF")
    ax.set_title(title or "Variance Inflation Factor (VIF)")
    ax.axvline(threshold, linestyle="--")
    # optional annotate numeric values
    for yi, v in zip(y, vals):
        ax.text(v, yi, f" {v:.2f}", va="center", ha="left")
    plt.tight_layout()
    plt.show()
    return df_plot

# ---------------- Example usage (hooks into your existing helper) ----------------
# formula = "r ~ C(weekday) + ToM + C(month) + log_size + C(sector)"
# vif_tbl = compute_vif_from_formula(formula, df)   # from your MLR-Practice helper
# plot_vif_bar(vif_tbl, threshold=5.0, title="VIF (calendar-dummy OLS)")
